# Lab | Making predictions with logistic regression

**In this lab, you will be using the Sakila database of movie rentals.**

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

**Instructions**

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.
2. Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.
3. Read the data into a Pandas dataframe.
4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict this variable from the cleaned data.
Evaluate the results.

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass 

In [14]:
password = getpass.getpass()

········


In [15]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

With some help from Ferreira I managed to import the cells I want. I adapted his SQL code to import film id, times a movie was rented in total, True/False whether it was rented in May, and how often that month, rental duration, length, rating, category and rental rate.

I did not include information such as extra features (I don't believe they would matter much), Language (they are all English anyways), Title (we are judging by factors like category & rating, length, price and historical demand and have the id if we need to find out titles)

In [22]:
query = '''SELECT f.film_id, COUNT(r.rental_id) AS times_rented, f.rental_duration, f.length,
f.rating, c.name AS category, f.rental_rate, 
CASE
    WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN True
    ELSE False END AS may
FROM sakila.film f
Left JOIN inventory i
    ON f.film_id = i.film_id
JOIN sakila.rental r
    ON i.inventory_id = r.inventory_id
Join sakila.film_category fc
    On fc.film_id = f.film_id
Join sakila.category c
    On c.category_id = fc.category_id
WHERE r.rental_date BETWEEN '2005-01-01' AND '2005-12-31'
GROUP BY film_id, rental_duration, f.length, f.rating, category, may, f.rental_rate;'''

In [23]:
film = pd.read_sql(query, engine)
film.head()

,film_id,times_rented,rental_duration,length,rating,category,rental_rate,may
0,19,19,6,113,PG,Action,0.99,0
1,19,1,6,113,PG,Action,0.99,1
2,21,2,3,129,R,Action,4.99,1
3,21,19,3,129,R,Action,4.99,0
4,29,10,5,168,NC-17,Action,2.99,0


In [ ]:
film = pd.read_sql(query, engine)
film